In [1]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import pandas as pd
import numpy as np
import pickle, csv
from datetime import datetime, timedelta

instrument = "EUR_USD"
params = {
    "from": "2018-01-01T22:00:00Z",
    "to": "2021-03-09T22:00:00Z",
    "granularity": "H4",
    "count": 5000,
}
directory = 'training/v3/_1/'
client = oandapyV20.API(access_token='199d815eb1f94c176a86dd3d5cc991a5-5e6ff7a15c524bf65fc0af5ae9a11972')
fields = ['time', 'open', 'high', 'low', 'close']
def get_data(instrument, params, filename, client):
    reset_file(filename)
    for i in InstrumentsCandlesFactory(instrument=instrument, params=params):
        rv = client.request(i)
        data_to_csv(filename, i.response)
    return
def data_to_csv(file_name, r):
    with open(file_name, mode='a+') as file:
        writer = csv.DictWriter(file, fieldnames=fields)
        for candle in r.get('candles'):
            values = {
                'time': candle['time'][:19]+'Z',
                'open': candle['mid']['o'],
                'high': candle['mid']['h'],
                'low': candle['mid']['l'],
                'close': candle['mid']['c'],
            }
            writer.writerow(values)
    return
def reset_file(file_name):
    file = open(file_name, 'w')
    file.truncate(0)
    file.close()
    with open(file_name, mode='a+') as file:
        writer = csv.DictWriter(file, fieldnames=fields)
        writer.writeheader()
    return
get_data(instrument, params, "h4_candles.csv", client)
h4 = pd.read_csv('h4_candles.csv')
h4[['open', 'high', 'low', 'close']] = h4[['open', 'high', 'low', 'close']].astype(np.float64)

x=h4.at[167, 'time']
params['from'] = x[:11]+str(int(x[11:13])+2)+x[13:]
params['granularity'] = 'M15'
get_data(instrument, params, "m15_candles.csv", client)
m15 = pd.read_csv("m15_candles.csv")
m15[['open', 'high', 'low', 'close']] = m15[['open', 'high', 'low', 'close']].astype(np.float64)

In [2]:
def trends(df):
    df['short_term_trend'] = (df['close']>df['ema-50']).astype(int)
    df['mid_term_trend'] = (df['close']>df['ema-100']).astype(int)
    df['long_term_trend'] = (df['close']>df['ema-200']).astype(int)
def time_of_day_week(df):
    df['time_of_day'] = df.loc[:, 'time'].str[11:13]
    df['time_of_day'] = df['time_of_day'].astype(int)
    Y=df.loc[:, 'time'].str[:4]
    M=df.loc[:, 'time'].str[5:7]
    D=df.loc[:, 'time'].str[8:10]
    df['day_of_week'] = [datetime(year=int(y), month=int(m), day=int(d)).weekday() for y, m, d in zip(Y, M, D)]
def cci(df, period=20):
    df['TP'] = (df['high']+df['low']+df['close'])/3
    df['TP_SMA']=0.0
    df['CCI']=0.0
    for i in range(period, len(df)):
        df.at[i, 'TP_SMA'] = np.mean(df.loc[i-period:i, 'TP'])
        mean_deviation = np.mean(np.abs(df.at[i, 'TP_SMA'] - df.loc[i-period:i, 'TP']))
        df.at[i, 'CCI'] = (df.at[i, 'TP'] - df.at[i, 'TP_SMA'])/(1.5 * mean_deviation)
def stochastic(df, period=14): # same as %K
    df['%K'] = 0.0
    for i in range(period, len(df)):
        high = np.max(df.loc[i-period:i, 'high'])
        low = np.min(df.loc[i-period:i, 'low'])
        close = df.at[i, 'close']
        df.at[i, '%K'] = (close-low)/(high-low)
def rsi(df, period=14):
    df['RSI'], df['mean_gain'], df['mean_loss'] = 0.0, 0.0, 0.0
    gain, loss = 0, 0
    for j in range(0, period):
        change = df.at[j+1, 'close']/df.at[j, 'close']
        if change>=1:
            gain += change-1
        else:
            loss += 1-change
    df.at[period, 'mean_gain'] = gain/period
    df.at[period, 'mean_loss'] = loss/period
    RS = gain/loss
    df.at[period, 'RSI'] = 1 - 1/(1+RS)
    for i in range(period+1, len(df)):
        gain, loss = 0, 0
        change = df.at[i, 'close']/df.at[i-1, 'close']
        if change>=1:
            gain += change-1
        else:
            loss += 1-change
        df.at[i, 'mean_gain'] = (df.at[i-1, 'mean_gain']*13 + gain)/period
        df.at[i, 'mean_loss'] = (df.at[i-1, 'mean_loss']*13 + loss)/period
        RS = df.at[i, 'mean_gain']/df.at[i, 'mean_loss']
        df.at[i, 'RSI'] = 1 - 1/(1+RS)
def ema(df, period):
    s = 'ema-{}'.format(period)
    df[s]=0.0
    df.at[period, s] = np.mean(df.loc[:period, 'close'])
    multiplier = 2/(1+period)
    for i in range(period+1, len(df)):
        df.at[i, s] = df.at[i, 'close']*multiplier + df.at[i-1, s]*(1-multiplier)
def atr(df, period=14):
    df['ATR']=0.0
    df['TR']=0.0
    for i in range(1, len(df)):
        h, l, c = df.at[i, 'high'], df.at[i, 'low'], df.at[i-1, 'close']
        df.at[i, 'TR'] = np.max([h-l, abs(h-c), abs(l-c)])
    for i in range(period, len(df)):
        df.at[i, 'ATR'] = np.mean(df.loc[i-period+1:i+1, 'TR'])
def add_indicators(df):
    cci(df)
    stochastic(df)
    rsi(df)
    for i in [12, 26, 50, 100, 200]:
        ema(df, i)
    df['macd'] = df['ema-12']-df['ema-26']
    s = 'signal'
    df[s]=0.0
    multiplier = 2/(1+9)
    df.at[40, s] = np.mean(df.loc[31:40, 'ema-12']-df.loc[31:40, 'ema-26'])
    for i in range(41, len(df)):
        df.at[i, s] = (df.at[i, 'ema-12']-df.at[i, 'ema-26'])*multiplier + df.at[i-1, s]*(1-multiplier)
    atr(df)
    df.drop(['TR', 'mean_gain', 'mean_loss'], axis=1, inplace=True)
    time_of_day_week(df)
    trends(df)

In [3]:
add_indicators(m15)
m15=m15[200:]
m15 = np.array(m15)
m15[:, 1:] = m15[:, 1:].astype(np.float64)
m15[:, [15, 16, 17]] *= 1000
m15[:, [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14]] = (m15[:, [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14]]-1)*10

h4 = np.array(h4)
h4[:, 1:] = h4[:, 1:].astype(np.float64)

In [4]:
from collections import deque
x1, x2, x3, y = deque(), deque(), deque(), deque()
j=180
stoploss=10e-3
spread=1.5e-3
for i in range(len(m15)-288-80):
    if m15[i, 0]==h4[j+1, 0]:
        j+=1
    x1.append(h4[j-180:j, 1:5])
    x2.append(m15[i:i+288, 1:5])
    x3.append(m15[i+287, 5:])

    leny = len(y)
    cp = m15[i+287, 4]
    k=30e-3
    short, long = False, False
    for l in range(288, 288+80):
        diff_low = cp-m15[i+l, 3]
        diff_high = cp-m15[i+l, 2]
        if not short and diff_low>stoploss-spread:
           short=True
        elif not long and diff_high<-stoploss+spread:
            long=True
        elif long and short:
            y.append([0, 1, 0])
            break
        elif long and diff_high<-k-spread:
            y.append([0, 0, 1])
            break
        elif short and diff_low>k+spread:
            y.append([1, 0, 0])
            break
    if len(y)!=leny+1:
        y.append([0, 1, 0])


x1=np.array(x1).astype(np.float64)
x2=np.array(x2).astype(np.float64)
x3=np.array(x3).astype(np.float64)
y=np.array(y).astype(np.float64)

In [5]:
np.sum(y, axis=0)

array([14404., 47753., 13757.])

In [6]:
from sklearn.model_selection import train_test_split
x1t, x1T, x2t, x2T, x3t, x3T, yt, yT = train_test_split(x1, x2, x3, y, test_size=0.3, shuffle=False)

In [7]:
t = np.sum(yt, axis=0)
ts = np.sum(t)
sample_weights = np.ones(len(yt)) * t[1]*2/ts
sample_weights[np.logical_or(yt[:, 0]>0, yt[:, 2]>0)] = (t[0]+t[2])*2/ts

In [8]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras import optimizers
from matplotlib import pyplot as plt
from IPython.display import clear_output

def model_init():
    input1 = Input(shape=(180, 4))
    x = LSTM(180)(input1)
    x = Dropout(0.2)(x)
    branch1 = Model(inputs=input1, outputs=x)

    input2 = Input(shape=(288, 4))
    y = LSTM(288)(input2)
    y = Dropout(0.2)(y)
    branch2 = Model(inputs=input2, outputs=y)

    input3 = Input(shape=18)
    a = Dense(128, activation=keras.layers.LeakyReLU(alpha=0.1))(input3)
    a = Dropout(0.2)(a)
    features_branch = Model(inputs=input3, outputs=a)

    combined = concatenate([branch1.output, branch2.output, features_branch.output])#, features_branch.output])
    # z = Dense(256, activation=tf.keras.layers.LeakyReLU(alpha=0.1))(combined)
    # z = Dropout(0.2)(z)
    # z = Dense(3, activation="softmax", name='10_pip')(z)
    #
    # b = Dense(256, activation=tf.keras.layers.LeakyReLU(alpha=0.1))(combined)
    # b = Dropout(0.2)(b)
    # b = Dense(3, activation="softmax", name='20_pip')(b)

    c = Dense(256, activation=keras.layers.LeakyReLU(alpha=0.1))(combined)
    c = Dropout(0.2)(c)
    c = Dense(3, activation="softmax", name='30_pip')(c)

    model = Model(inputs=[branch1.input, branch2.input, features_branch.input], outputs=c)#, outputs=[z, b, c])
    adam = optimizers.Adam(learning_rate=0.0005)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['categorical_accuracy'], run_eagerly=False)
    return model

class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 1
        self.x = []
        self.losses = []
        self.val_losses = []

        self.fig = plt.figure()

        self.logs = []

    def on_epoch_end(self, epoch, logs={}):

        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1

        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show()

plot_losses = PlotLosses()

In [9]:
path = 'models/_9_added_time_and_trends/m'
from keras.callbacks import ModelCheckpoint
model=model_init()
model.call = tf.function(model.call, experimental_relax_shapes=True)

checkpoint = ModelCheckpoint(filepath=path+'.{epoch:02d}',
                 monitor='val_loss',
                 verbose=1,
                 save_best_only=False,
                save_weights_only=True)

In [ ]:
model.fit(x=[x1t, x2t, x3t], y=yt, batch_size=1000, epochs=100, validation_split=0.1,
          shuffle=True, callbacks=[plot_losses, checkpoint],
          sample_weight=sample_weights,
          initial_epoch=50)

In [58]:
model.load_weights(path+'.46') # 50, 46, 38, 40

In [ ]:
pt = model([x1t, x2t, x3t], training=False)
pT = model([x1T, x2T, x3T], training=False)

In [70]:
net_profit_pips, i = 0, 0
winning, losing = [], []
candles, pred = x2T, pT
neutral=0

horizon=500
spread=1.5e-3
takeprofit=30e-3
stop = 10e-3
cutoff=0.2

highest_preloss=[]
# 0.35 for 5 pip, 0.25 for 3 pip
while i<len(candles)-horizon:
    stoploss=stop
    cp = candles[i, -1, 3]
    ep = candles[i+horizon, -1, 3]
    diff=0
    closed=False
    if pred[i, 0]>cutoff or pred[i, 2]>cutoff:
        long = pred[i, 2]>cutoff
        if long:
            cp+=spread/2
        else:
            cp-=spread/2
        for l in range(1, 1+horizon):
            diff_low = cp-candles[i+l, -1, 2]
            diff_high = cp-candles[i+l, -1, 1]
            if long and diff_low>stoploss-spread/2:
                diff = -stoploss
                if stoploss>0:
                    highest_preloss.append((np.max(candles[i+1:i+l+1, -1, 1])-cp)*1e3)
                i+=l
                closed=True
                break
            elif not long and diff_high<-stoploss+spread/2:
                diff = -stoploss
                if stoploss>0:
                    highest_preloss.append((cp-np.min(candles[i+1:i+l+1, -1, 2]))*1e3)
                i+=l
                closed=True
                break
            # elif long and diff_high<-stoploss-spread/2:
            #     stoploss=0
            # elif not long and diff_low>stoploss+spread/2:
            #     stoploss=0
            if long and diff_high<-takeprofit-spread/2:
                diff = takeprofit
                i+=l
                closed=True
                break
            elif not long and diff_low>takeprofit+spread/2:
                diff = takeprofit
                i+=l
                closed=True
                break
        if not closed:
            diff = ep-cp if long else cp-ep
            i+=horizon
        diff*=1e3
        net_profit_pips+=diff
        if diff>0:
            winning.append(diff)
        elif diff==0:
            neutral+=1
        else:
            losing.append(diff)
    i+=1

s = len(winning)+len(losing)
print(net_profit_pips, np.mean(winning), np.mean(losing), s, neutral)
print(len(winning), len(losing), len(winning)/(len(winning)+len(losing)))
print(np.quantile(highest_preloss, 0.7))

-550.0 30.0 -10.0 551 0
124 427 0.2250453720508167
9.290000000000905
